In [1]:
import sys
import os
import numpy as np
import cv2 
import albumentations as aug
import pandas as pd
from random import randint
import random


In [3]:
image_path = 'id'
back_path = 'background'


class augmentor():
    def __init__(self):
        
         self.aug={}
        
         self.aug['hf']=aug.Compose([aug.HorizontalFlip(p=1)], keypoint_params=aug.KeypointParams(format='xy'))
         self.aug['vf']=aug.Compose([aug.VerticalFlip(p=1)], keypoint_params=aug.KeypointParams(format='xy'))
         self.aug['bc']=aug.Compose([aug.RandomBrightnessContrast(p=1,brightness_limit=0.2, contrast_limit=0.2)], keypoint_params=aug.KeypointParams(format='xy'))
         self.aug['jc']=aug.Compose([aug.JpegCompression (quality_lower=10, quality_upper=100, always_apply=False, p=0.5)], keypoint_params=aug.KeypointParams(format='xy'))
         self.aug['bl']=aug.Compose([aug.GaussianBlur (blur_limit=(1, 1), sigma_limit=2.5, always_apply=False, p=0.5)], keypoint_params=aug.KeypointParams(format='xy'))
         self.aug['mb']=aug.Compose([aug.MotionBlur(p=0.2)], keypoint_params=aug.KeypointParams(format='xy'))
         self.aug['rg']=aug.Compose([aug.augmentations.transforms.RandomGamma(gamma_limit=(80, 120))], keypoint_params=aug.KeypointParams(format='xy'))
         self.aug['cl']=aug.Compose([aug.augmentations.transforms.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5)], keypoint_params=aug.KeypointParams(format='xy'))  
         self.aug['ip']=aug.Compose([aug.imgaug.transforms.IAAPerspective (scale=(0.05, 0.1), keep_size=True, always_apply=False, p=0.5) ,aug.RandomBrightnessContrast(p=1,brightness_limit=0.5, contrast_limit=0.4)], keypoint_params=aug.KeypointParams(format='xy'))  
         self.aug['ig']=aug.Compose([aug.imgaug.transforms.IAAAdditiveGaussianNoise (loc=0, scale=(2.5500000000000003, 12.75), per_channel=False, always_apply=False, p=0.5),aug.RandomBrightnessContrast(p=1,brightness_limit=0.5, contrast_limit=0.4)], keypoint_params=aug.KeypointParams(format='xy'))  
         self.aug['rc']=aug.augmentations.transforms.RandomResizedCrop (512, 512, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1, always_apply=False, p=1.0) 
         self.aug['sf']=aug.Compose([aug.augmentations.transforms.RandomSunFlare (flare_roi=(0, 0, 1, 0.5))], keypoint_params=aug.KeypointParams(format='xy'))  
         self.aug['sh']=aug.Compose([aug.augmentations.transforms.RandomShadow (shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5)], keypoint_params=aug.KeypointParams(format='xy'))  
         self.aug['rg']=aug.Compose([aug.augmentations.transforms.RandomGamma(gamma_limit=(80, 120), always_apply=False, p=0.5)], keypoint_params=aug.KeypointParams(format='xy'))  


    def __call__(self,image,keypoints,augment="hf"):
        kpts=[]
        if(augment=="rc"):
            augmented=self.aug[augment](image=image)
            return augmented['image']
            
        while(len(kpts)!=4):
            augmented=self.aug[augment](image=image,keypoints=keypoints)
            kpts=augmented['keypoints']    
          
        return augmented['image'],augmented['keypoints']
    
    
class Synthetic_generator():
    
    def __init__(self,image_path,back_path):
        
        self.imp=image_path
        self.bgp=back_path
        self.filters=['hf','vf','bc','jc','bl','mb','rg','cl','ip','ig','sf','sh','rg']
        self.filt_weigh=[5,5,20,5,10,5,1,5,20,5,10,10,10]
       
        self.augment=augmentor()
        
        
    def __getpoints(self,pts,pos):

        (yp,xp)=pos        
        points=pts+np.array([xp,yp])
        
        return points
        
        
    def __getpos(self,fore,back):
      
        f_size=fore.shape
        b_size=back.shape
        
        assert(b_size[0]>f_size[0] and b_size[1]>f_size[1]) 
        x1=f_size[0]
        x2=b_size[0]
        
        y1=f_size[1]
        y2=b_size[1]
        
        x=randint(0,(x2-x1))
        y=randint(0,(y2-y1))
       
        return (x,y)
    
    
    def get_filter(self):
         return random.choices(self.filters, weights=self.filt_weigh, k=1)[0]
         
            
    def stack_image(self,imgb,img2,pos):
       
        posx,posy=(pos)
        img1=imgb.copy()
        (w,h,c)=img2.shape
       
        crp=img1[posx:posx+w,posy:posy+h,:]
        crp[np.where(img2>=0)]=img2[np.where(img2>=0)].ravel()
        img1[posx:posx+w,posy:posy+h,:]=crp 
        
        return img1
    
    
    def save_image(self,img,name): 
        path = 'data/'
        cv2.imwrite(os.path.join(path ,name+'.jpg'), img)
        
    
    def get_dataframe(self,img_count,pts,path):
        
        [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]=pts
        name='image_'+str(img_count)+'.jpg'
        dataframe = pd.DataFrame([[path+name,  x1, y1, x2, y2, x3, y3, x4, y4]])
        
        return dataframe
    
    
    def crop_with_kp(self,image,kp):
       
        pts=np.array((kp), np.int32)
        
        minx=min(pts[:,0])
        maxx=max(pts[:,0])
        miny=min(pts[:,1])
        maxy=max(pts[:,1])
    
        y,x,_=image.shape
        xl=randint(0,minx)
        xr=randint(maxx,x-1)
        yu=randint(0,miny)
        yd=randint(maxy,y-1)
        new_img=image[yu:yd,xl:xr,:]
        pts=pts-np.array(([xl,yu]))
    
        return new_img,pts
    

    def generate_image(self,fore_img,back_img,f_size_ran,b_size_ran,rot_prob=1,randomcrop=False):

            fore=os.path.join(self.imp, fore_img)
            back=os.path.join(self.bgp, back_img)
            
            fore=cv2.imread(fore)
            back=cv2.imread(back)
            
          
            xf=randint(f_size_ran[0],f_size_ran[1])
            yf=int(fore.shape[0]/fore.shape[1]*xf)
            
            x=randint(int(xf*2),b_size_ran[1])
            y=randint(int(yf*2),b_size_ran[1])
        
            f_size=(xf,yf)
            b_size=(x,y)
           
            if(randomcrop):                
                back=self.augment(back,0,augment='rc')

            
            fore=cv2.resize(fore,f_size,interpolation = cv2.INTER_AREA)
            back=cv2.resize(back,b_size,interpolation = cv2.INTER_AREA)
            
            
            (yf,xf,c)=fore.shape
            pts=np.array(([(0,0),(xf,0),(xf,yf),(0,yf)]))# initial points
            
            
            if(randint(0,rot_prob)==0):# rotate matrix
                angle=random.randint(0, 360)
                fore=fore.astype("float32")
                fore=fore+1
                fore,pts=rotate(fore,angle,pts)
                fore=fore-1
    
            pos=self.__getpos(fore,back)
            new_img=self.stack_image(back,fore,pos)              
            point=self.__getpoints(pts, pos)
                                                      
            augimg,augpts=self.augment(new_img, point,augment=self.get_filter())
               
            img,pts=self.crop_with_kp(augimg,augpts)

            return img,pts
            
            
            
    def generate(self,f_size_ran,b_size_ran,rot_prob=1,randomcrop=False, path ='data/'):
        foreimage=os.listdir(self.imp)
        backimage=os.listdir(self.bgp)
        dataframe=pd.DataFrame()
        misimg=0
        img_count=0
        total=0
        

        for bimg in backimage:

            for fimg in foreimage : 

                try:
                   
                    img,pts=self.generate_image(fimg,bimg,f_size_ran,b_size_ran,rot_prob=rot_prob,randomcrop=randomcrop)
                    img_count+=1
                    
                    df=self.get_dataframe(img_count,pts,path)
                    
                    dataframe=dataframe.append(df,ignore_index=True)
                    img_name='image_'+str(img_count)
                    
                    self.save_image(img,img_name)
                   
                except :
                    misimg+=1

                finally:
                    total+=1
          
            
        dataframe.to_csv('synthetic_data.csv', header=['image_path', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])
        print("processed ",total,"samples\nmissed  ",misimg,"  samples")    


def rotate(img, angle, points):
    
        height, width = img.shape[:2] 
        image_center = (width/2, height/2) 
        
        ###-----box-----####

        ones = np.ones(shape=(len(points), 1))
        points_ones = np.hstack([points, ones])

        
        rotation_img = cv2.getRotationMatrix2D(image_center, angle, 1.)
        transformed_points = rotation_img.dot(points_ones.T).T
    
        pts=np.array((transformed_points), np.int32)
        pts[:,0]-=min(pts[:,0])
        pts[:,1]-=min(pts[:,1])
        
        ###---image---#####
        
        abs_cos = abs(rotation_img[0,0]) 
        abs_sin = abs(rotation_img[0,1])

        bound_w = int(height * abs_sin + width * abs_cos)
        bound_h = int(height * abs_cos + width * abs_sin)

        rotation_img[0, 2] += bound_w/2 - image_center[0]
        rotation_img[1, 2] += bound_h/2 - image_center[1]


        rotated_img = cv2.warpAffine(img, rotation_img, (bound_w, bound_h))
        
        return rotated_img,pts
        
       

              
    
args=sys.argv

gen=Synthetic_generator(image_path,back_path)

gen.generate((300,390),(1000,1024),randomcrop=True)




processed  25 samples
missed   0   samples
